## 2.10 Tensorflow Datasets
In order to use a Dataset we need three steps:
1. **Importing Data** : Create a Dataset instance from some data  
2. **Create an Iterator** : By using the created dataset to make an Iterator instance to iterate thought the dataset
3. **Consuming Data** : By using the created iterator we can get the elements from the dataset to feed the model  

#### Generating data from numpy
This is the common case, we have a numpy array and we want to pass it to tensorflow.

In [3]:
import tensorflow as tf;
# create a random vector of shape (100,2)
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

In [5]:
data_iter = dataset.make_one_shot_iterator()
dt = data_iter.get_next()

In [11]:
sess =tf.Session()
with sess.as_default():
    print (sess.run(dt))
    print (sess.run(dt))
    

[0.8775927  0.50346294]
[0.35036019 0.01597385]


In [12]:
with sess.as_default():
    print (sess.run(dt))
    print (sess.run(dt))
    

[0.21268736 0.93003424]
[0.4789219  0.63771947]


## Generate Dataset from Placeholders
This is useful when we want to dynamic change the data inside the Dataset, we will se later how.

In [14]:
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

In [15]:
# from generator
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])
def generator():
    for el in sequence:
        yield el
dataset = tf.data.Dataset().batch(1).from_generator(generator,
                                           output_types= tf.int64, 
                                           output_shapes=(tf.TensorShape([None, 1])))
iter = dataset.make_initializable_iterator()
el = iter.get_next()
with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))
    print(sess.run(el))
    print(sess.run(el))

[[1]]
[[2]
 [3]]
[[3]
 [4]
 [5]]
